# Funky Radical Electrodynamics Engine 1105 #

This Project (which can be abbreviated to FREE1105) Will Exist in 5 Phases:

1) Set Up the Electric Fields in a 2D Coordinate System
2) Simulate the Motion of Electric Point Charges
3) Simluate the Motion of Charged Rigid Bodies
4) Visualise the Field via Geometrical Curvature
5) Implement Maxwell's 4th Law for Magnetic Fields and Light

Bismillah Ar Rahman Ar Rahim - In Sha Allah this will go well!!

## Phase 1 ##

Let us establish the checklist of aims:
- Set Up the window to initialise a screen
- Add a wall on the screen so as to not have particles go so far off screen
- Be able to add particles on the 2d Screen to form point charges with initial positions
- Implement these point charges to be able to be static, environmental boolean values
- Implement the necassary equations for superpositions of electric fields
- Visualise the electric field in 2d when static as a heatmap in the baackground colour

Points of Refinement After
- implement a reset button to eliminate all non environmental charges and take everything back to inintial position and initial velocity
- implement a right click and drop down menu to then place components like point charges and then to drag them around

Alright so In Sha Allah lets do this and vaguely follow some vague classical approximation of physics in the process!!

#### Planning the Architecture of Phase 1 ####

We start with 5 files:
- the main pygame file
- constants imported for all files
- point charge class information
- electric field file
- gui file

Rough Plan:

1) The main pygame loop will load all the imports, initialise a screen and then set up a wall entity and throw it on there on the boundaries of the screen
2) write in the constants file that is imported for everything
2) Then in the point charge file, I will make a point charge class with attributes charge, mass, core_radius, border_radius, total_radius, position, pos_0, position, vel_0, velocity, environmental, static
3) Then create a function to relatively determine core_radius in comparison to other charges and a function to do the same for the border radius with mass. Then total radius is their sum
4) Create the electric field calculator and heatmap. Create a loop going through every pixel of a 1/4 scale virtual map and scale upwards to the full screen where in each pixel, the total net field at that space is calculated. 
5) heatmap is produced where effectively the background colour changes from default grey. at highest positive charge, it is blackest and at lowest negative charge, it is whitest. The Scale of charge will be symmetrical, so if the highest + charge is like idk 20 and highest negative is like 5, the 5 will be only 25% between grey and white whereas the posiitve will be fully black
6) make the gui file and make the reset button as a box in the bottom left as an object of its own class
7) make a drop down menu show up when you do a right click with for now just point charge. Then you click on the button and it opens a box containing options for its creation, being charge, mass, environmental and static toggles and the initial position is where the right click was
8) allow the point charge to get right clicked on, with options to drag to another position for now and allowing you to update the position.

#### How it Actually Went 1.0 ####

First off, I vibecoded the hell outta the GUI file. Pygame GUI isnt that tricky but oh my word, I genuinely couldn't care less forcing myself to make a class for toggles and text boxes. Otherwise it went good. I consulted Gemini on how to best implement the background screen so I could represent the electric field and it said to do a virtual screen and scale up 4x for 16x better performance. I tinkered with the colours a lot and yippee, I am happy with the end results.

Several times I vibe coded the GUI, lost complete track of what was going on and just like gave up and started again. I HATE GUI STUFF I SUCK. THE MATHS AND PHYSICS IS EASY. MICROMANAGING THE PIXEL DISTANCES SO IT DOES NOT LOOK LIKE GARBAGE SUCKS!! 

In [ ]:
%%writefile gui_jypter1.py

# below is constants_for_all_files.py

# necessary imports

import pygame
import numpy as np

# dimensions

SW = 1500 # Screen Width
SH = 900  # Screen Height
# (1/4 scale for E-Field calc) then scale up for perfomance
V_SW = SW // 4 # Virtual Width 
V_SH = SH // 4 # Virtual Height 

# default colors + wall stuff

BGC = 128, 128, 128  # Background Color for Neutral Grey
WC = (0,0, 0) # Neon Green for Wall Color
text_c = (30,30, 30) # Dark Grey for Text Color
WALL_RECT = pygame.Rect(0, 0, SW, SH)

# physics constants

WBT = 20 # Wall Border Thickness
BW_coeff = 0.9 # Coefficient of Restitution for Bouncing Wall
K_COULOMB = 8.99e9 # Coulomb's Constant in N m²/C²
E_CHARGE = 1.602e-19 # Elementary Charge in Coulombs
E_MASS = 9.109e-31 # Electron Mass in kg
P_MASS = 1.672e-27 # Proton Mass in kg

In [ ]:
%%writefile gui_jypter1.py

import pygame
import numpy as np
from constants_for_all_files import *

# --- UI CONFIGURATION ---
FONT_NAME = 'Consolas'
FONT_SIZE = 16
COL_BG = (240, 240, 240)
COL_BORDER = (0, 0, 0)
COL_ACTIVE = (50, 150, 255) # Blue highlight
COL_INACTIVE = (150, 150, 150)
COL_TEXT = (0, 0, 0)

# Standard UI Colors (ON/OFF)
COL_TRUE_STD = (50, 200, 50)    # Green
COL_FALSE_STD = (200, 50, 50)   # Red

# Physics Colors (Positive/Negative)
COL_POS = (200, 0, 0)   # Red
COL_NEG = (0, 0, 200)   # Blue

class ResetButton:
    def __init__(self):
        # Bottom Left Placement
        self.rect = pygame.Rect(20, SH - 60, 100, 40)
        self.font = pygame.font.SysFont('Arial', 20, bold=True)
        self.hovered = False

    def handle_event(self, event):
        if event.type == pygame.MOUSEMOTION:
            self.hovered = self.rect.collidepoint(event.pos)
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.hovered:
                return True
        return False

    def render(self, screen):
        color = (200, 50, 50) if self.hovered else (150, 50, 50)
        pygame.draw.rect(screen, color, self.rect)
        pygame.draw.rect(screen, (0,0,0), self.rect, 2)
        
        text = self.font.render("RESET", True, (255, 255, 255))
        text_rect = text.get_rect(center=self.rect.center)
        screen.blit(text, text_rect)

class StartButton:
    def __init__(self):
        # Placed next to ResetButton
        self.rect = pygame.Rect(1380, SH - 60, 100, 40)
        self.font = pygame.font.SysFont('Roboto', 24, bold=True)
        self.hovered = False
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEMOTION:
            self.hovered = self.rect.collidepoint(event.pos)
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.hovered:
                return True
        return False

    def render(self, screen):
        # Green Color for Start/Run
        color = (50, 150, 50) if self.hovered else (50, 100, 50)
        pygame.draw.rect(screen, color, self.rect)
        pygame.draw.rect(screen, (0,0,0), self.rect, 2)
        
        text = self.font.render("START", True, (255, 255, 255))
        text_rect = text.get_rect(center=self.rect.center)
        screen.blit(text, text_rect)

class WallCORSlider:
    """Slider for wall coefficient of restitution (0.0 to 1.0)"""
    def __init__(self, x, y, width, height, initial_value=0.9):
        self.rect = pygame.Rect(x, y, width, height)
        self.slider_rect = pygame.Rect(x, y + height // 2 - 5, width, 10)
        self.knob_radius = 8
        self.value = initial_value
        self.dragging = False
        self.font = pygame.font.SysFont('Arial', 14)
        self.label = "Wall Bounce:"
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            knob_x = self.slider_rect.x + self.value * self.slider_rect.width
            knob_rect = pygame.Rect(knob_x - self.knob_radius, self.slider_rect.y - self.knob_radius, 
                                   self.knob_radius * 2, self.knob_radius * 2)
            if knob_rect.collidepoint(event.pos):
                self.dragging = True
                
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            self.dragging = False
            
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            # Update value based on mouse position
            relative_x = event.pos[0] - self.slider_rect.x
            self.value = max(0.0, min(1.0, relative_x / self.slider_rect.width))
            
    def render(self, screen):
        # Draw label
        label_surf = self.font.render(self.label, True, (0, 0, 0))
        screen.blit(label_surf, (self.rect.x, self.rect.y - 20))
        
        # Draw track
        pygame.draw.rect(screen, (200, 200, 200), self.slider_rect)
        pygame.draw.rect(screen, (0, 0, 0), self.slider_rect, 1)
        
        # Draw filled portion (progress bar)
        filled_width = self.slider_rect.width * self.value
        filled_rect = pygame.Rect(self.slider_rect.x, self.slider_rect.y, filled_width, self.slider_rect.height)
        pygame.draw.rect(screen, (100, 150, 255), filled_rect)
        
        # Draw knob
        knob_x = self.slider_rect.x + self.value * self.slider_rect.width
        knob_color = (50, 100, 200) if self.dragging else (100, 150, 255)
        pygame.draw.circle(screen, knob_color, (int(knob_x), int(self.slider_rect.centery)), self.knob_radius)
        pygame.draw.circle(screen, (0, 0, 0), (int(knob_x), int(self.slider_rect.centery)), self.knob_radius, 2)
        
        # Draw value text
        value_text = self.font.render(f"{self.value:.2f}", True, (0, 0, 0))
        screen.blit(value_text, (self.slider_rect.right + 10, self.slider_rect.centery - 7))

class ContextMenu:
    def __init__(self):
        self.active = False
        self.pos = (0, 0)
        self.rect = pygame.Rect(0, 0, 160, 30)
        self.font = pygame.font.SysFont('Arial', 16)

    def show(self, pos):
        self.active = True
        self.pos = pos
        self.rect.topleft = pos

    def handle_event(self, event):
        if not self.active: return None
        
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.rect.collidepoint(event.pos):
                return "open_creator" # Clicked "Point Charge"
            else:
                self.active = False # Clicked away
        return None

    def render(self, screen):
        if not self.active: return
        pygame.draw.rect(screen, (240, 240, 240), self.rect)
        pygame.draw.rect(screen, (0,0,0), self.rect, 1)
        text = self.font.render("Create Charge", True, (0,0,0))
        screen.blit(text, (self.rect.x + 10, self.rect.y + 5))

# --- TEXT/TOGGLE COMPONENTS ---

class ScientificTextBox:
    """A text box that strictly accepts scientific notation inputs."""
    def __init__(self, x, y, w, h, default_text="1.0e-6"):
        self.rect = pygame.Rect(x, y, w, h)
        self.text = default_text
        self.active = False
        self.font = pygame.font.SysFont(FONT_NAME, FONT_SIZE)
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.active = True
            else:
                self.active = False
                
        if event.type == pygame.KEYDOWN and self.active:
            if event.key == pygame.K_RETURN:
                self.active = False
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                # Strict filter: Digits, ., -, +, e, E
                if event.unicode in "0123456789.-+eE":
                    self.text += event.unicode

    def get_value(self):
        """Returns float if valid, None otherwise."""
        try:
            return float(self.text)
        except ValueError:
            return None

    def render(self, screen):
        # Border color indicates focus
        color = COL_ACTIVE if self.active else COL_INACTIVE
        pygame.draw.rect(screen, (255, 255, 255), self.rect)
        pygame.draw.rect(screen, color, self.rect, 2)
        
        # Text rendering
        text_surf = self.font.render(self.text, True, COL_TEXT)
        # Center text vertically
        text_rect = text_surf.get_rect(midleft=(self.rect.x + 5, self.rect.centery))
        screen.blit(text_surf, text_rect)

class MantissaTextBox:
    """Text box that enforces x.y format (max 2 digits before decimal)."""
    def __init__(self, x, y, w, h, default_text="1.0"):
        self.rect = pygame.Rect(x, y, w, h)
        self.text = default_text
        self.active = False
        self.font = pygame.font.SysFont(FONT_NAME, FONT_SIZE)
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.active = True
            else:
                self.active = False
                
        if event.type == pygame.KEYDOWN and self.active:
            if event.key == pygame.K_RETURN:
                self.active = False
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                # Allow digits and one decimal point
                if event.unicode in "0123456789.":
                    # Prevent multiple decimal points
                    if event.unicode == "." and "." in self.text:
                        return
                    # Prevent more than 2 digits before decimal
                    if event.unicode.isdigit():
                        parts = self.text.split(".")
                        if len(parts[0]) >= 2:
                            return
                    self.text += event.unicode

    def get_value(self):
        """Returns float if valid, None otherwise."""
        try:
            val = float(self.text)
            return val if val > 0 else None
        except ValueError:
            return None

    def render(self, screen):
        color = COL_ACTIVE if self.active else COL_INACTIVE
        pygame.draw.rect(screen, (255, 255, 255), self.rect)
        pygame.draw.rect(screen, color, self.rect, 2)
        
        text_surf = self.font.render(self.text, True, COL_TEXT)
        text_rect = text_surf.get_rect(midleft=(self.rect.x + 5, self.rect.centery))
        screen.blit(text_surf, text_rect)

class ExponentTextBox:
    """Text box that enforces integer exponent from -99 to 99."""
    def __init__(self, x, y, w, h, default_text="0"):
        self.rect = pygame.Rect(x, y, w, h)
        self.text = default_text
        self.active = False
        self.font = pygame.font.SysFont(FONT_NAME, FONT_SIZE)
        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.active = True
            else:
                self.active = False
                
        if event.type == pygame.KEYDOWN and self.active:
            if event.key == pygame.K_RETURN:
                self.active = False
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                # Allow digits and one minus sign at the start
                if event.unicode in "0123456789-":
                    # Minus sign only at start and only one
                    if event.unicode == "-":
                        if len(self.text) == 0 or self.text[0] == "-":
                            if len(self.text) == 0:
                                self.text = "-"
                        return
                    # Limit to 2 digits (+ potential minus sign)
                    max_len = 3 if self.text.startswith("-") else 2
                    if len(self.text) < max_len:
                        self.text += event.unicode

    def get_value(self):
        """Returns float if valid, None otherwise."""
        try:
            val = int(self.text)
            return float(val) if -99 <= val <= 99 else None
        except ValueError:
            return None

    def render(self, screen):
        color = COL_ACTIVE if self.active else COL_INACTIVE
        pygame.draw.rect(screen, (255, 255, 255), self.rect)
        pygame.draw.rect(screen, color, self.rect, 2)
        
        text_surf = self.font.render(self.text, True, COL_TEXT)
        text_rect = text_surf.get_rect(midleft=(self.rect.x + 5, self.rect.centery))
        screen.blit(text_surf, text_rect)

class BinaryToggle:
    """A generic toggle button with customizable colors and text."""
    def __init__(self, x, y, w, h, label, initial_state=True, 
                 text_true="ON", text_false="OFF", 
                 col_true=COL_TRUE_STD, col_false=COL_FALSE_STD):
        
        self.rect = pygame.Rect(x, y, w, h)
        self.label = label
        self.state = initial_state
        self.font = pygame.font.SysFont('Arial', 14, bold=True)
        
        # Custom Configuration
        self.text_true = text_true
        self.text_false = text_false
        self.col_true = col_true
        self.col_false = col_false

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.rect.collidepoint(event.pos):
                self.state = not self.state
                return True
        return False

    def render(self, screen):
        # Determine Color and Text based on state
        if self.state:
            bg_col = self.col_true
            status_txt = self.text_true
        else:
            bg_col = self.col_false
            status_txt = self.text_false
            
        pygame.draw.rect(screen, bg_col, self.rect)
        pygame.draw.rect(screen, (0,0,0), self.rect, 2)
        
        # Status Text inside button
        txt_surf = self.font.render(status_txt, True, (255,255,255))
        txt_rect = txt_surf.get_rect(center=self.rect.center)
        screen.blit(txt_surf, txt_rect)
        
        # Label Text above button
        if self.label:
            label_surf = self.font.render(self.label, True, (0,0,0))
            screen.blit(label_surf, (self.rect.x, self.rect.y - 18))

class CreationForm:
    def __init__(self):
        self.active = False
        self.rect = pygame.Rect(0, 0, 380, 360)
        self.target_pos = (0,0)
        self.font = pygame.font.SysFont('Arial', 16, bold=True)
        self.edit_mode = False  # False = Create, True = Edit
        self.editing_particle = None  # Reference to particle being edited
        
        # --- COMPONENTS ---
        
        # 1. Remember Toggle (Standard Green/Red)
        self.btn_remember = BinaryToggle(0, 0, 120, 25, "Remember Input", False)
        
        # 2. Charge Input & Sign Toggle (CUSTOM RED/BLUE)
        self.btn_charge_sign = BinaryToggle(0, 0, 50, 30, "", True, 
                                            text_true="+", text_false="-",
                                            col_true=COL_POS, col_false=COL_NEG)
        
        self.input_charge_mantissa = MantissaTextBox(0, 0, 80, 30, "1.0")
        self.input_charge_exponent = ExponentTextBox(0, 0, 60, 30, "0")
        
        # 3. Mass Input
        self.input_mass_mantissa = MantissaTextBox(0, 0, 80, 30, "1.0")
        self.input_mass_exponent = ExponentTextBox(0, 0, 60, 30, "0")
        
        # 4. Environmental & Static Toggles (Standard Green/Red)
        self.btn_env = BinaryToggle(0, 0, 60, 30, "Env", True)
        self.btn_static = BinaryToggle(0, 0, 60, 30, "Static", True)
        
        # 5. Action Buttons
        self.btn_submit = pygame.Rect(0,0, 100, 30)
        self.btn_cancel = pygame.Rect(0,0, 100, 30)

    def show(self, pos, particle=None):
        """Show form. If particle is provided, enter edit mode."""
        self.active = True
        self.target_pos = pos
        self.edit_mode = particle is not None
        self.editing_particle = particle
        self.rect.center = (SW//2, SH//2)
        
        if self.edit_mode and particle:
            # Load particle's current values into form
            # Decompose charge into mantissa and exponent
            charge_val = abs(particle.charge)
            if charge_val == 0:
                c_mantissa, c_exponent = 1.0, 0
            else:
                import math
                c_exponent = int(math.floor(math.log10(charge_val)))
                c_mantissa = charge_val / (10 ** c_exponent)
            
            # Decompose mass into mantissa and exponent
            mass_val = particle.mass
            if mass_val == 0:
                m_mantissa, m_exponent = 1.0, 0
            else:
                import math
                m_exponent = int(math.floor(math.log10(mass_val)))
                m_mantissa = mass_val / (10 ** m_exponent)
            
            self.input_charge_mantissa.text = f"{c_mantissa:.1f}"
            self.input_charge_exponent.text = str(c_exponent)
            self.input_mass_mantissa.text = f"{m_mantissa:.1f}"
            self.input_mass_exponent.text = str(m_exponent)
            self.btn_charge_sign.state = particle.charge >= 0
            self.btn_env.state = particle.environmental
            self.btn_static.state = particle.static
        else:
            # Create mode: reset to defaults
            self.input_charge_mantissa.text = "1.0"
            self.input_charge_exponent.text = "0"
            self.input_mass_mantissa.text = "1.0"
            self.input_mass_exponent.text = "0"
            self.btn_charge_sign.state = True
            self.btn_env.state = True
            self.btn_static.state = True
            
        # --- LAYOUT CALCULATION ---
        rx, ry = self.rect.x, self.rect.y
        
        # Remember Input (Top Right)
        self.btn_remember.rect.topright = (self.rect.right - 20, ry + 30)
        
        # Charge Row (Moved down to Y=90 for label clearance)
        self.btn_charge_sign.rect.topleft = (rx + 40, ry + 90)
        self.input_charge_mantissa.rect.topleft = (rx + 100, ry + 90)
        self.input_charge_exponent.rect.topleft = (rx + 230, ry + 90)
        
        # Mass Row (Moved down to Y=160)
        self.input_mass_mantissa.rect.topleft = (rx + 100, ry + 160)
        self.input_mass_exponent.rect.topleft = (rx + 230, ry + 160)
        
        # Toggles Row
        self.btn_env.rect.topleft = (rx + 50, ry + 230)
        self.btn_static.rect.topleft = (rx + 200, ry + 230)
        
        # Buttons
        self.btn_submit.bottomleft = (rx + 30, self.rect.bottom - 20)
        self.btn_cancel.bottomright = (self.rect.right - 30, self.rect.bottom - 20)

    def handle_event(self, event):
        if not self.active: return None

        # Pass events to children
        self.input_charge_mantissa.handle_event(event)
        self.input_charge_exponent.handle_event(event)
        self.input_mass_mantissa.handle_event(event)
        self.input_mass_exponent.handle_event(event)
        self.btn_remember.handle_event(event)
        self.btn_charge_sign.handle_event(event)
        self.btn_env.handle_event(event)
        self.btn_static.handle_event(event)

        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            # Check Submit
            if self.btn_submit.collidepoint(event.pos):
                
                # Validate Inputs
                c_mantissa = self.input_charge_mantissa.get_value()
                c_exponent = self.input_charge_exponent.get_value()
                m_mantissa = self.input_mass_mantissa.get_value()
                m_exponent = self.input_mass_exponent.get_value()
                
                print(f"DEBUG: c_mantissa={c_mantissa}, c_exponent={c_exponent}, m_mantissa={m_mantissa}, m_exponent={m_exponent}")
                
                # Validation: All must be numbers, mantissas in range [1.0-9.9], mass > 0
                if (c_mantissa is not None and c_exponent is not None and 
                    m_mantissa is not None and m_exponent is not None and
                    1.0 <= c_mantissa <= 9.9 and 1.0 <= m_mantissa <= 9.9):
                    
                    # Construct charge value: mantissa * 10^exponent
                    charge = c_mantissa * (10 ** c_exponent)
                    if not self.btn_charge_sign.state: # If state is False (Blue/-)
                        charge = -charge
                    
                    # Construct mass value: mantissa * 10^exponent
                    mass = m_mantissa * (10 ** m_exponent)
                    
                    print(f"DEBUG: Validation passed. charge={charge}, mass={mass}")
                    
                    if mass > 0:
                        self.active = False
                        result = {
                            "charge": charge,
                            "mass": mass,
                            "environmental": self.btn_env.state,
                            "static": self.btn_static.state
                        }
                        
                        if self.edit_mode:
                            # Edit mode: return particle reference and edited flag
                            result["particle"] = self.editing_particle
                            result["edit"] = True
                        else:
                            # Create mode: return position for new particle
                            result["position"] = np.array(self.target_pos, dtype=float)
                        
                        return result
            
            # Check Cancel
            elif self.btn_cancel.collidepoint(event.pos):
                self.active = False
                return None
                
        return None

    def render(self, screen):
        if not self.active: return
        
        # Draw Panel
        pygame.draw.rect(screen, COL_BG, self.rect)
        pygame.draw.rect(screen, COL_BORDER, self.rect, 2)
        
        # Title (different based on mode)
        title_text = "Edit Charge" if self.edit_mode else "Create Charge"
        title = self.font.render(title_text, True, COL_TEXT)
        screen.blit(title, (self.rect.x + 20, self.rect.y + 20))
        
        # Labels for TextBoxes
        l_chg = self.font.render("Charge:", True, COL_TEXT)
        screen.blit(l_chg, (self.rect.x + 100, self.input_charge_mantissa.rect.top - 20))
        
        l_mass = self.font.render("Mass:", True, COL_TEXT)
        screen.blit(l_mass, (self.rect.x + 100, self.input_mass_mantissa.rect.top - 20))
        
        # x 10^ labels
        exp_label_c = self.font.render("x 10 ^", True, COL_TEXT)
        screen.blit(exp_label_c, (self.input_charge_mantissa.rect.right + 10, self.input_charge_mantissa.rect.centery - 8))
        
        exp_label_m = self.font.render("x 10 ^", True, COL_TEXT)
        screen.blit(exp_label_m, (self.input_mass_mantissa.rect.right + 10, self.input_mass_mantissa.rect.centery - 8))
        
        # Render Components
        self.btn_remember.render(screen)
        self.btn_charge_sign.render(screen)
        self.input_charge_mantissa.render(screen)
        self.input_charge_exponent.render(screen)
        self.input_mass_mantissa.render(screen)
        self.input_mass_exponent.render(screen)
        self.btn_env.render(screen)
        self.btn_static.render(screen)
        
        # Render Action Buttons
        # Submit (Green-ish)
        pygame.draw.rect(screen, (100, 200, 100), self.btn_submit)
        pygame.draw.rect(screen, (0,0,0), self.btn_submit, 1)
        submit_text = "CHANGE" if self.edit_mode else "SPAWN"
        ts = self.font.render(submit_text, True, (0,0,0))
        ts_rect = ts.get_rect(center=self.btn_submit.center)
        screen.blit(ts, ts_rect)
        
        # Cancel (Red-ish)
        pygame.draw.rect(screen, (200, 100, 100), self.btn_cancel)
        pygame.draw.rect(screen, (0,0,0), self.btn_cancel, 1)
        tc = self.font.render("CANCEL", True, (0,0,0))
        tc_rect = tc.get_rect(center=self.btn_cancel.center)
        screen.blit(tc, tc_rect)

In [ ]:
%%writefile point_charges_jypter1.py

# below is point_charges.py

# necessary imports

import pygame 
import numpy as np
from constants_for_all_files import *

# Point Charge Class

class PointCharge:

    def __init__(self, pos_0, charge, mass, environmental, static, pc_id):
        self.position = np.array(pos_0, dtype=float) # Position Vector
        self.vel = np.array([0.0, 0.0], dtype=float) # Velocity Vector
        self.pos_0 = np.array(pos_0, dtype=float) # Initial Position Vector
        self.vel_0 = np.array([0.0, 0.0], dtype=float) # Initial Velocity Vector
        self.charge = charge # Charge in Coulombs
        self.mass = mass # Mass in kg
        self.environmental = environmental # Boolean for Environmental Charge
        self.static = static # Boolean for Static Charge
        self.pc_id = pc_id # Unique ID for Point Charge
        self.dragging = False # Boolean for Dragging State

        # Initialize radius values (will be updated by update_relative_scale)
        self.core_radius = 15  # default core radius
        self.border_radius = 6  # default border radius
        self.total_radius = self.core_radius + self.border_radius

        self.color = (200, 0, 0) if charge > 0 else (0, 0, 200) # Red for Positive, Blue for Negative

    @property
    def pos(self):
        """Property to always return current position (for compatibility with electric_field.py)."""
        return self.position

    def update_relative_scale(self, all_charges):
        
        # Determines core_radius based on charge magnitude vs others.
        # Determines border_radius based on mass vs others.
        
        if not all_charges:
            return

        # 1. Calculate Core Radius (Charge)
        max_q = max(abs(pc.charge) for pc in all_charges)
        min_q = min(abs(pc.charge) for pc in all_charges)
        q_range = max_q - min_q
        
        current_q = abs(self.charge)
        
        if q_range == 0:
            norm_q = 0.5
        else:
            # (current - min) / range -> 0 to 1
            norm_q = (current_q - min_q) / q_range
            
        self.core_radius = 5 + (norm_q * 15) # Scale 5px to 20px

        # 2. Calculate Border Radius (Mass)
        max_m = max(pc.mass for pc in all_charges)
        min_m = min(pc.mass for pc in all_charges)
        m_range = max_m - min_m
        
        current_m = self.mass
        
        if m_range == 0:
            norm_m = 0.5
        else:
            norm_m = (current_m - min_m) / m_range
            
        self.border_radius = 1 + (norm_m * 4) # Scale 1px to 5px
        
        # 3. Sum for total hitbox
        self.total_radius = self.core_radius + self.border_radius

    def reset(self):

        # reverts to initial state
        
        if not self.environmental:
            self.pos = self.pos_0.copy()
            self.vel = self.vel_0.copy()
        else:
            # delete the point charge if NOT environmental
            del self
    
    def render(self, screen):
        # Draw Border (Mass representation) - Colored based on charge
        if self.charge > 0:
            border_colour_fill = (220, 90, 90)  # Orange for Positive Charge Border
        elif self.charge < 0:
            border_colour_fill = (130, 20, 130)  # Purple for Negative Charge Border
        else:
            border_colour_fill = (100, 100, 100)  # Grey for neutral
            
        pygame.draw.circle(screen, border_colour_fill, self.position.astype(int), int(self.total_radius))
        
        # Draw Core (Charge representation) - Colored
        pygame.draw.circle(screen, self.color, self.position.astype(int), int(self.core_radius))
        
        # Selection Highlight
        if self.dragging:
            pygame.draw.circle(screen, (255, 255, 255), self.position.astype(int), int(self.total_radius + 2), 2)
            # sets new position while dragging
            self.position = np.array(pygame.mouse.get_pos(), dtype=float)

In [ ]:
%%writefile electric_field_jypter1.py

# below is electric_field.py file

# necessary imports

import pygame
import numpy as np
from constants_for_all_files import *

# Electric Field Calculation and Visualization

class ElectricFieldSystem:

    def __init__(self):

        # sets up the virtual grid

        self.x_coords = np.linspace(0, SW, V_SW)
        self.y_coords = np.linspace(0, SH, V_SH)
        self.grid_x, self.grid_y = np.meshgrid(self.x_coords, self.y_coords)
        
        # surface to hold the heatmap

        self.surface = pygame.Surface((V_SW, V_SH))

    def render(self, target_surface, charges):

        if not charges:
            target_surface.fill(BGC)
            return

        # Calculate the electric field at each grid point due to all charges

        net_field = np.zeros_like(self.grid_x, dtype=float)

        for pc in charges:
            dx = self.grid_x - pc.pos[0]
            dy = self.grid_y - pc.pos[1]
            r2 = dx**2 + dy**2
            r2[r2 == 0] = E_CHARGE # Prevent division by zero
            min_distance = pc.total_radius ** 2
            r2[r2 < min_distance] = min_distance # Cap minimum distance squared to avoid extreme values

            contribution = (K_COULOMB * pc.charge) / r2
            net_field += contribution

        # colourises the heatmap based on field strength

        max_val = np.max(np.abs(net_field))
        if max_val == 0:
            max_val = 100  # so eveything is neutral grey

        normalised = 0.7 * net_field / max_val
        pixel_vals = 127.5 - (normalised * 127.5)
        pixel_vals = np.clip(pixel_vals, 0, 255).astype(np.uint8)
        
        # create RGB array for Pygame surface

        pixels = np.dstack([pixel_vals] * 3)
        pygame.surfarray.blit_array(self.surface, np.swapaxes(pixels, 0, 1))
        
        # scale up to full screen

        scaled_view = pygame.transform.scale(self.surface, (SW, SH))
        target_surface.blit(scaled_view, (0, 0))
            

In [ ]:
%%writefile main_jypter1.py

# below is the main.py file

# necessary imports

import pygame
import numpy as np
import sys

# import all other necessary modules

from constants_for_all_files import *
from point_charges import *
from electric_field import *
from gui import *

# set up the window

pygame.init()
screen = pygame.display.set_mode((SW, SH))
pygame.display.set_caption("FREE1105: Phase 1")
clock = pygame.time.Clock()

# setting up systems

ef_system = ElectricFieldSystem()
reset_btn = ResetButton()
context_menu = ContextMenu()
create_form = CreationForm()
start_btn = StartButton()
wall_cor_slider = WallCORSlider(30, 20, 150, 30, initial_value=BW_coeff)

# setting up initial state of the simulation

all_point_charges = []
selected_charge = None
p_id_counter = 0
wall_cor = wall_cor_slider.value  # Current wall coefficient of restitution

# the main loop where everything happens

running = True
while running:

    # event handling

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
            sys.exit()

        # handle reset button

        if reset_btn.handle_event(event):
            all_point_charges = [pc for pc in all_point_charges if pc.environmental]
            for pc in all_point_charges:
                pc.reset()

        # handle start button

        if start_btn.handle_event(event):
            pass  # TODO: Start simulation logic

        # handle wall COR slider

        wall_cor_slider.handle_event(event)

        # handle context menu

        menu_action = context_menu.handle_event(event)
        if menu_action == "open_creator":
            create_form.show(context_menu.pos)

        # creation/edit form handling

        form_data = create_form.handle_event(event)
        if form_data:
            try:
                if form_data.get('edit'):
                    # Edit mode: update existing particle
                    particle = form_data['particle']
                    particle.charge = form_data['charge']
                    particle.mass = form_data['mass']
                    particle.environmental = form_data['environmental']
                    particle.static = form_data['static']
                    # Update color based on new charge sign
                    particle.color = (200, 0, 0) if particle.charge > 0 else (0, 0, 200)
                    particle.update_relative_scale(all_point_charges)
                    print(f"EDIT: Particle updated - charge={particle.charge}, mass={particle.mass}, env={particle.environmental}, static={particle.static}")
                else:
                    # Create mode: spawn new particle
                    new_pc = PointCharge(
                        pos_0=form_data['position'],
                        charge=form_data['charge'],
                        mass=form_data['mass'],
                        environmental=form_data['environmental'],
                        static=form_data['static'],
                        pc_id=p_id_counter
                    )
                    all_point_charges.append(new_pc)
                    p_id_counter += 1
            except Exception as e:
                import traceback
                print("Error in form submission:")
                print("form_data=", form_data)
                traceback.print_exc()

        # mouse interactions
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1: # Left Click (Drag)
                if not context_menu.active and not create_form.active:
                    for pc in reversed(all_point_charges):
                        # Check distance for click
                        dist = np.linalg.norm(pc.position - np.array(event.pos))
                        if dist < pc.total_radius:
                            selected_charge = pc
                            pc.dragging = True
                            break
                            
            elif event.button == 3: # Right Click (Menu or Edit)
                clicked_on_charge = False
                clicked_charge = None
                for pc in reversed(all_point_charges):
                     dist = np.linalg.norm(pc.position - np.array(event.pos))
                     if dist < pc.total_radius:
                         # Right clicked a charge - open edit form
                         clicked_on_charge = True
                         clicked_charge = pc
                         break
                
                if clicked_on_charge and clicked_charge:
                    # Open edit form for this charge
                    create_form.show(event.pos, particle=clicked_charge)
                else:
                    # Open context menu to create new charge
                    context_menu.show(event.pos)

        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1 and selected_charge:
                selected_charge.dragging = False
                selected_charge = None
                
        elif event.type == pygame.MOUSEMOTION:
            if selected_charge:
                selected_charge.position = np.array(event.pos, dtype=float)

    # logic and updates

    wall_cor = wall_cor_slider.value  # Update wall coefficient from slider

    for pc in all_point_charges:
        pc.update_relative_scale(all_point_charges)

    # rendering

    ef_system.render(screen, all_point_charges)
    pygame.draw.rect(screen, WC, WALL_RECT, WBT) # draw wall
    for pc in all_point_charges:
        pc.render(screen)

    # gui rendering

    reset_btn.render(screen)
    start_btn.render(screen)
    wall_cor_slider.render(screen)
    context_menu.render(screen)
    create_form.render(screen)

    pygame.display.flip()
    clock.tick(60)  # Limit to 60 FPS

pygame.quit()
sys.exit()

In [ ]:
from main_jypter1 import *

## Phase 2 ##

Let us establish the checklist of aims:
- When the Start button is pressed, stop allowing user interaction, unless reset, with only a pause button allowed
- Implement motion of the point charges from completely static spawn and when the reset button is clicked, the sim is stopped and they all go back to their initial positions or get OBLITERATED INTO OBLIVION if not self.environmental. And then you can drag and drop stuff for the sim
- Establish after effects and then after trails like in angry birds that disappear in like 10-30s idk
- Establish collision mechanics between particles and the wall with its sliding coefficient meter. I MAY implement variable coefficient of restitutions as an ability to create and change point charges
- Then finally implement some form of variable initial velocity. Exact details idk. Then implement an arrow indicator on the object before the simulation is run to indicate the direction and relative size. May make the arrow draggable to establish direction with more precision once in preview mode. But when the sim is running, its gone

Points for Refinement After:
- Time intervals and playback times will have to undergo significant scaling for the purpose of usability and visuals. 
- May experiment regarding how to visually represent the coefficient of restitution for things

#### Planning the Architecture of Phase 2 ####

We start with 5 files from Phase 1, All will likely be augumented:
- the main pygame file
- constants imported for all files
- point charge class information
- electric field file
- gui file

The Following new Files will be made:
- 

Rough Plan:

1) 